In [32]:
import numpy as np
import torch
from torch.utils import data

In [33]:
def synthetic_data(w, b, num_of_examples):
    """
    生成 y = Xx + b + 噪声
    :param num_of_examples:
    :param w: 真实的权重
    :param b: 真实的偏差
    :return: 数据集features，标签label
    """
    X = torch.normal(0, 1, (num_of_examples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)

# 调用框架中现有的API来读取数据

In [34]:
def load_array(data_arrays, batch_size, is_train=True):
    """
    构造一个PyTorch数据迭代器
    :param data_arrays:数组（features，labels）
    :param batch_size:批量大小b
    :param is_train:是否训练
    :return:iterator
    """
    dataset = data.TensorDataset(*data_arrays)  # 得到一个dataset
    return  data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

next(iter(data_iter))

[tensor([[ 0.3137,  1.2574],
         [ 1.1746,  0.4877],
         [ 0.6695,  1.1772],
         [-1.2151, -0.6242],
         [-1.2482,  1.1550],
         [-0.4736, -0.9863],
         [ 0.9058, -2.1846],
         [ 0.9985, -0.4747],
         [ 0.5647,  1.1515],
         [-0.6004, -1.0954]]),
 tensor([[ 0.5631],
         [ 4.8984],
         [ 1.5326],
         [ 3.8940],
         [-2.2192],
         [ 6.6155],
         [13.4384],
         [ 7.8184],
         [ 1.4078],
         [ 6.7067]])]

# 使用框架的预定义好的层

In [35]:
from torch import nn  # nn 是神经网络的缩写

net = nn.Sequential(nn.Linear(2, 1))  # 输入2维，输出1维

# 初始化模型参数

In [36]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

In [37]:
loss = nn.MSELoss()  # 均方误差
trainer = torch.optim.SGD(net.parameters(), lr=0.03)  # 实列化SGD

# 开始训练

In [38]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)  # w和b已经设置好
        trainer.zero_grad()  # 梯度清零
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000181
epoch 2, loss 0.000101
epoch 3, loss 0.000099
